In [1]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
import io
import shutil

from __future__ import print_function

import os.path

In [22]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/documents.readonly', 'https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']

# The ID of a sample document.
DOCUMENT_ID = '1JztVntjVBFpG-FBJzFp5z5RxSJtLtqf4wwls2hmSh0w'

In [3]:
def main():
    """Shows basic usage of the Docs API.
    Prints the title of a sample document.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return creds

creds=main()

In [68]:
text=''

In [23]:
try:
    service = build('docs', 'v1', credentials=creds)

    # Retrieve the documents contents from the Docs service.
    document = service.documents().get(documentId=DOCUMENT_ID).execute()

    print('The title of the document is: {}'.format(document.get('title')))
    # print(document.get('body'))
        
except HttpError as err:
    print(err)

The title of the document is: J017_Yash_Chauhan_MPSTME_new


In [24]:
for i in document.get('body')['content']:
    if 'paragraph' in i.keys():
        if 'bullet' in i['paragraph'].keys():
            print(i['paragraph'])

{'elements': [{'startIndex': 260, 'endIndex': 340, 'textRun': {'content': 'Current CGPA of 3.84 over the course of VI Semesters with major in Data Science.', 'textStyle': {'bold': False, 'italic': False, 'underline': False, 'strikethrough': False, 'smallCaps': False, 'backgroundColor': {}, 'foregroundColor': {'color': {'rgbColor': {}}}, 'fontSize': {'magnitude': 10, 'unit': 'PT'}, 'weightedFontFamily': {'fontFamily': 'Times New Roman', 'weight': 400}, 'baselineOffset': 'NONE'}}}, {'startIndex': 340, 'endIndex': 341, 'textRun': {'content': '\n', 'textStyle': {'bold': False, 'italic': False, 'underline': False, 'strikethrough': False, 'smallCaps': False, 'backgroundColor': {}, 'foregroundColor': {'color': {'rgbColor': {}}}, 'fontSize': {'magnitude': 10, 'unit': 'PT'}, 'weightedFontFamily': {'fontFamily': 'Noto Sans Symbols', 'weight': 400}, 'baselineOffset': 'NONE'}}}], 'paragraphStyle': {'namedStyleType': 'NORMAL_TEXT', 'alignment': 'START', 'lineSpacing': 100, 'direction': 'LEFT_TO_RIG

In [20]:
import json

with open('data.json', 'w') as fp:
    json.dump(document.get('body'), fp)

In [73]:
print(text)

Raj Kiran                                                                                                                                                       Email: rajkirann92@gmail.com
Phone: 8163492618
                                                                                         Azure DataOps Architect

A dedicated and innovative IT professional having experiences in Development, Servers Management, DevSecOps and Security. 10+ Years IT experience as a Cloud DevOps Engineer. Expert in developing, managing and troubleshooting On premise and Cloud (AWS, Azure) based Virtual machines (Linux /Windows), Data Pipelines , Containerised (Kubernetes, OpenShift) Infrastructure. Experience in developing APIs, microservices,  serverless functions using Python , NodeJS, Java.  Strong Linux background fueled my ability to implement, automate and optimize the Continuous Integration (CI) and Continuous Deployment (CD) pipelines in DevOps methodology.

SUMMARY:

Hands-on experience in de

In [37]:
requests = [
    ### adding text (last point endIndex+1)
    # {
    #     'insertText': {
    #         'location': {
    #             'index': 2640,
    #         },
    #         'text': 'Python insertion \n'
    #     }
    # },
    ### deleting text (startIndex-1)
    # {
    #         'deleteContentRange': {
    #             'range': {
    #                 'startIndex': 2427,
    #                 'endIndex': 2640,
    #             }

    #         }

    # }
    ### creating bullet points
    # {
    #     'createParagraphBullets': {
    #         'range': {
    #             'startIndex': 2640,
    #             'endIndex':  2640
    #         },
    #         'bulletPreset': 'BULLET_DISC_CIRCLE_SQUARE',
    #     }
    # }
    ### removing bullet points
    # {
    #     'deleteParagraphBullets': {
    #         'range': {
    #             'startIndex': 2893,
    #             'endIndex':  3117
    #         },
    #     }
    # }

]

try:
    service = build('docs', 'v1', credentials=creds)
    result = service.documents().batchUpdate(
        documentId=DOCUMENT_ID, body={'requests': requests}).execute()
except HttpError as e:
    print(e)

In [28]:
def download_file(real_file_id, creds, filename):
    """Downloads a file
    Args:
        real_file_id: ID of the file to download
    Returns : IO object with location.

    Load pre-authorized user credentials from the environment.
    TODO(developer) - See https://developers.google.com/identity
    for guides on implementing OAuth2 for the application.
    """
    try:
        # create drive api client
        service = build('drive', 'v3', credentials=creds)

        file_id = real_file_id

        # pylint: disable=maybe-no-member
        request = service.files().export_media(fileId=file_id, mimeType='application/vnd.openxmlformats-officedocument.wordprocessingml.document')
        file = io.BytesIO()
        downloader = MediaIoBaseDownload(file, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            # print(F'Download {int(status.progress() * 100)}. Done: {done}')

    except HttpError as error:
        print(F'An error occurred: {error}')
        file = None

    print(file)
    file.seek(0)
    with open(filename, 'wb') as f:
        shutil.copyfileobj(file, f)

    # return file

# https://docs.google.com/document/d/17sfw7Ifjxh6h5Tina0dM--8AAA-ZTefyQVc_7vwyVXo/edit?usp=drive_link
download_file(real_file_id=DOCUMENT_ID, creds=creds, filename='./downloaded.docx')

In [33]:
def uploadFile(filename, uploadname):
    service = build('drive', 'v3', credentials=creds)

    file_metadata = {
    'name': uploadname,
    'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'
    }
    media = MediaFileUpload(filename ,
                            mimetype='application/vnd.openxmlformats-officedocument.wordprocessingml.document',
                            resumable=True)
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print ('File ID: ' + file.get('id'))

uploadFile('downloaded.docx', 'uploaded.docx')

File ID: 1Ejtv9pa8XDg5I4wnvuqW4e-inyXelfoB
